In [1]:
import pandas as pd
# Constructs a .CSV as input for PointConnector plugin for QGIS


In [2]:
# work to create a dict class
dict_plants = dict()


In [ ]:
# Concept that will be used to make vertices and edges of directed graph
for i in range(len(df_plants)):
    pName = str(df_plants.iloc[i,2])
    dict_plants[pName] = list()
#print(len(dict_plants.keys()))
for k in dict_plants.keys():
    dict_plants[k].extend([1])
    print(str(k))

In [59]:
# df_plants.columns.get_loc("Name")
# df_plants.reset_index().head()
# df_plants[df_plants['Name'] == 501].iloc[0]
df_plants.loc[df_plants['Name'] == 501,'Name']
# df_plants.head()

Series([], Name: Name, dtype: int64)

In [3]:
#Load the images and plant csv's

# matched plantID with Image Coords, "Bens" means i manually labeled several points believed to be a standard
df_matched =  pd.read_csv("..\\CSV\\SourceSheets\\Bens_WORKFILE_ST_07.csv")#("WORKFILE_ST_07.csv") 
# the plant id and coords
df_plants = pd.read_csv('..\\CSV\\SourceSheets\\LeafLocations.csv')
# The angle calculated from each time-ordered subsequent point:
df_imgWangle = pd.read_csv('..\\CSV\\CalculatedSheets\\AvgCoordinates_Degrees2.csv')
# the point where a picture was taken
# df_image_points = pd.read_csv('IOP3_July2021_image_gps.csv')
df_image_pointsAVG = pd.read_csv('..\\CSV\\CalculatedSheets\\TripodAvgGPS1.csv') 

In [25]:
df_imgWangle.head()

,img_id,latitude,longitude,degree
0,tripod_canopy_2020_210727_144827,38.456153,-122.412243,-2.713581
1,tripod_canopy_2020_210727_144949,38.456156,-122.412236,0.482830
2,tripod_canopy_2020_210727_145105,38.456149,-122.412248,-1.758535
3,tripod_canopy_2020_210727_145248,38.456425,-122.412195,-2.586930
4,tripod_canopy_2020_210727_145449,38.456576,-122.411951,-2.701490


In [4]:
df_image_pointsAVG.head()

,img_id,latitude,longitude
0,tripod_canopy_2020_210729_112023,38.456326,-122.410642
1,tripod_canopy_2020_210729_112101,38.456352,-122.410558
2,tripod_canopy_2020_210729_112145,38.456341,-122.410575
3,tripod_canopy_2020_210729_112225,38.456262,-122.410726
4,tripod_canopy_2020_210729_112321,38.456215,-122.410792


In [ ]:
df_plants.head()

In [70]:
## Creates a joined spreadsheet of both images and plant locations
##
#df_plants.head()
df_joinP = df_plants[['Name','X','Y']]
#df_image_points.head()
df_joinI = df_image_points[['img_id','latitude','longitude']]
df_joinI['Name'] = df_joinI['img_id']
df_joinI['Y'] = df_joinI['latitude']
df_joinI['X'] = df_joinI['longitude']
df_joinI = df_joinI[['Name','X','Y']]
# df_joinI.rename(columns={"img_id":"Name","latitude":"Y","longitude":"X"})
bothLists = pd.concat([df_joinI,df_joinP])
bothLists.to_csv('Img_Plants.csv', index=False)



In [5]:
## This removes all the images of Standards, Only plants images remain
print(str(len(df_matched))+ "rows before std filter")

df_matched = df_matched[df_matched['Plant #']!= 'STD']
print(str(len(df_matched))+" rows after std filter")
df_matched.head()

463rows before std filter
415 rows after std filter


,Image,X,Y,DT,Unnamed: 4,Plant #,Unnamed: 6,Unnamed: 7
3,tripod_canopy_2020_210727_145248,-122.412193,38.456420,2021-07-27 14:52:49 PDT,NaN,501,NaN,NaN
4,tripod_canopy_2020_210727_145449,-122.411958,38.456578,2021-07-27 14:54:50 PDT,NaN,502,NaN,NaN
5,tripod_canopy_2020_210727_145546,-122.411795,38.456653,2021-07-27 14:55:50 PDT,NaN,503,NaN,NaN
6,tripod_canopy_2020_210727_145634,-122.411618,38.456755,2021-07-27 14:56:33 PDT,NaN,504,NaN,NaN
7,tripod_canopy_2020_210727_145727,-122.411370,38.456907,2021-07-27 14:57:27 PDT,NaN,505,NaN,NaN


In [6]:
## Creates a full dataframe for matching values
# Notes on df:
# 
df_imgWtime = df_matched.merge(df_image_pointsAVG,left_on='Image',right_on='img_id')
df_imgWtime = df_imgWtime.merge(df_imgWangle,left_on='Image',right_on='img_id',how='left')
df_imgWtime = df_imgWtime[['Image','X','Y','DT','degree']]
print("The length of the left df: "+str(len(df_matched)))
print("Length of right: "+str(len(df_image_points)))
print("Final length: "+str(len(df_imgWtime)))
df_imgWtime.head()
#df_imgWtime.to_csv('..\\CSV\\CalculatedSheets\\ImageLocation')
# This still needs the angle

The length of the left df: 415
Length of right: 464
Final length: 415


,Image,X,Y,DT,degree
0,tripod_canopy_2020_210727_145248,-122.412193,38.456420,2021-07-27 14:52:49 PDT,-2.586930
1,tripod_canopy_2020_210727_145449,-122.411958,38.456578,2021-07-27 14:54:50 PDT,-2.701490
2,tripod_canopy_2020_210727_145546,-122.411795,38.456653,2021-07-27 14:55:50 PDT,-2.589741
3,tripod_canopy_2020_210727_145634,-122.411618,38.456755,2021-07-27 14:56:33 PDT,-2.615097
4,tripod_canopy_2020_210727_145727,-122.411370,38.456907,2021-07-27 14:57:27 PDT,-2.704317


In [7]:
# df_imgWtime = df_imgWtime[['Image','X','Y','DT']]
df_imgWtime.to_csv('TripodAvgTimeAngle.csv',index=False)

In [41]:
# within df_matched, there currently exists 48 images marked as STD
# and 416 which are matches 
len(df_matched[df_matched['Plant #'] != 'STD'].values)

416

In [9]:


#import PlantLabel
df_imageData = df_matched[df_matched['Plant #'] != 'STD']


In [12]:
df_image_pointsAVG.head()

,img_id,latitude,longitude,test
0,tripod_canopy_2020_210729_112023,38.456326,-122.410642,39.456326
1,tripod_canopy_2020_210729_112101,38.456352,-122.410558,39.456352
2,tripod_canopy_2020_210729_112145,38.456341,-122.410575,39.456341
3,tripod_canopy_2020_210729_112225,38.456262,-122.410726,39.456262
4,tripod_canopy_2020_210729_112321,38.456215,-122.410792,39.456215


In [11]:
df_image_pointsAVG['test'] = df_image_pointsAVG['latitude'].apply(lambda x : x+1)

In [5]:
df_matched.head()

,Image,X,Y,DT,Unnamed: 4,Plant #,Unnamed: 6,Unnamed: 7
0,tripod_canopy_2020_210727_144827,-122.412240,38.456152,2021-07-27 14:48:26 PDT,NaN,STD,NaN,NaN
1,tripod_canopy_2020_210727_144949,-122.412235,38.456157,2021-07-27 14:49:51 PDT,NaN,STD,NaN,NaN
2,tripod_canopy_2020_210727_145105,-122.412248,38.456150,2021-07-27 14:51:06 PDT,NaN,STD,NaN,NaN
3,tripod_canopy_2020_210727_145248,-122.412193,38.456420,2021-07-27 14:52:49 PDT,NaN,501,NaN,NaN
4,tripod_canopy_2020_210727_145449,-122.411958,38.456578,2021-07-27 14:54:50 PDT,NaN,502,NaN,NaN


In [8]:
df_matched.iloc[1][1]

Image         tripod_canopy_2020_210727_144949
X                                  -122.412235
Y                                    38.456157
DT                     2021-07-27 14:49:51 PDT
Unnamed: 4                                 NaN
Plant #                                    STD
Unnamed: 6                                 NaN
Unnamed: 7                                 NaN
Name: 1, dtype: object

In [17]:
df_matched.iloc[1][1]

-122.412235

In [ ]:
df_matched['Plant #'].duplicated()

In [ ]:
df_image_points.count()

In [ ]:
df_matched.head()

In [ ]:
df_plants.count()

In [ ]:
# There appears to be a duplicate in plant name?
df_plants.where(df_plants['Name'].duplicated()).dropna()

In [ ]:
df_plants['Name'].where(df_plants['Name'].duplicated()).dropna()

In [ ]:
# *
#   Executing this cell displays work completed by matching leaf to drone image.
#   First, a process to remove all null fields.
# 
# 1. Collect all non-null fields  
#df_matched_p = df_matched[['Plant #','Image','X','Y']]
# 2. Change column name to correctly link to joining table
#df_matched_p['Name'] = df_matched_p['Plant #']
#df_matched_p = df_matched_p[['Name','Image','X','Y']]
# 3. Display
#df_matched_p.head(3)

In [ ]:
#
# # The first UNION PARAMETER
#
#
df_Plant_Locations = df_plants[['Name','X','Y']]
#df_Plant_Locations.to_csv('PlantLocationIDXY.csv',index = False)
df_Plant_Locations.head()

In [ ]:
#df_image_points.head()

In [ ]:
# 
#   The second UNION parameter
#
#df_image_points[['Image','X','Y']].to_csv('Image_GPS_XY.csv',index=False)
df_image_locations = df_image_points[['Image','X','Y']]
df_image_locations['Name'] = df_image_locations['Image']
df_image_locations = df_image_locations[['Name','X','Y']]   
df_image_locations.head(5)

In [ ]:
df_source_table = pd.concat([df_Plant_Locations,df_image_locations])
df_source_table.columns.value_counts()

In [ ]:
df_source_table.values

In [ ]:
#   This is the base layer that PointConnector will join on using the From, To table.
# df_source_table.to_csv('Leaf_AND_Image_Locations.csv',index=False)


In [ ]:
#Creates the From,To table parameter
FromTo = df_matched[['Image','Plant #']]
FromTo.head()
FromTo = FromTo[FromTo['Plant #']!='STD']
FromTo.value_counts()

In [ ]:
# Write as csv to use in QGIS
FromTo.to_csv('FromTo.csv',index=False)